In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/Heart Diseases.csv')

# Object Columns

thalack

In [ ]:
df.loc[65,'thalack'] = 142
df.loc[19,'thalack'] = 171
df['thalack'] = df['thalack'].astype(str).astype(int)
print(df.dtypes)

ID          object
age        float64
sex          int64
cp           int64
restbps      int64
chol         int64
fbs          int64
restecg      int64
thalack      int64
exang        int64
oldpeak    float64
slope        int64
ca          object
thal        object
outcome      int64
dtype: object


ca

In [ ]:
c = 0
for i,x in df.iterrows():
  if(x['ca'] == 1 and x['outcome'] == 0):
    c = c+1
print(c)

0


In [ ]:
df['ca'] = df['ca'].replace({'?':  df['outcome'].mode()[0]})
df['ca'] = df['ca'].astype(str).astype(int)

In [ ]:
print(df.dtypes)

ID          object
age        float64
sex          int64
cp           int64
restbps      int64
chol         int64
fbs          int64
restecg      int64
thalack      int64
exang        int64
oldpeak    float64
slope        int64
ca           int64
thal        object
outcome      int64
dtype: object


thal

In [ ]:
df[df['thal'] == '?']
df['thal'] = df['thal'].replace({'?':df['thal'].mode()[0]})

In [ ]:
df['thal'] = df['thal'].astype(str).astype(int)